In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('data\processed_data(by_quarter).csv')
df.head()

,Unnamed: 0,duration,goal_usd,blurb_length,name_length,usd_pledged,success,US based,main_category_comics,main_category_crafts,...,main_category_games,main_category_journalism,main_category_music,main_category_photography,main_category_publishing,main_category_technology,main_category_theater,start_Q_Q2,start_Q_Q3,start_Q_Q4
0,0,0.163043,0.000015,0.382353,0.230769,0.000705,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,0.315217,0.000030,0.676471,0.269231,0.000455,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,0.315217,0.000009,0.588235,0.230769,0.000129,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,0.478261,0.000027,0.411765,0.192308,0.000559,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,0.641304,0.000232,0.411765,0.115385,0.004696,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [3]:
df.drop(['Unnamed: 0', 'goal_usd'], axis = 1, inplace = True)
df.head()

,duration,blurb_length,name_length,usd_pledged,success,US based,main_category_comics,main_category_crafts,main_category_dance,main_category_design,...,main_category_games,main_category_journalism,main_category_music,main_category_photography,main_category_publishing,main_category_technology,main_category_theater,start_Q_Q2,start_Q_Q3,start_Q_Q4
0,0.163043,0.382353,0.230769,0.000705,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.315217,0.676471,0.269231,0.000455,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.315217,0.588235,0.230769,0.000129,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.478261,0.411765,0.192308,0.000559,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.641304,0.411765,0.115385,0.004696,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


## Develop XG Boost on data (with Pleage USD)

In [10]:
def do_xg(df, y_column,m_depth):

    from xgboost import XGBClassifier
    
  #1 : split x and y
    x = df.drop(labels= y_column,axis=1,)
    y = df[y_column]

  #2: split test and train
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x,y,train_size = 0.8, random_state = 42) 

  #3:Instantiate the estimator(classifier)
    xgmodel = XGBClassifier(use_label_encoder = False, max_depth= m_depth, n_jobs = -1 ) #n_jobs = -1 use all core to run the code 

  #4: fit the model
    xgmodel.fit(x_train, y_train, verbose = False)

  #5: Make a prediction
    y_pred =  xgmodel.predict(x_test)

  #6: caculate the acc score
    from sklearn.metrics import accuracy_score
    a_score = accuracy_score(y_test, y_pred) #y_test =  y_true
    #compare true score vs prediction score

  #7: caculate the proba #prediction use the question paper
    probability = xgmodel.predict_proba(x_test)

  #8: confusion matrix
    from sklearn.metrics import confusion_matrix
    c_matrix = confusion_matrix(y_test, y_pred)


  #9: classifier report
    from sklearn.metrics import classification_report
    c_report = classification_report(y_test, y_pred)

  #10: save the model, save as the JSON file
    filename = 'xgmodel' + str(m_depth) +'.json'
    xgmodel.save_model(filename)
    
    #11: KFolds cross validation
    from sklearn.model_selection import cross_val_score
    accuracy = cross_val_score(xgmodel, x_train, y_train, cv = 5)
    print ('KFolds cross validation: \n ', accuracy)
    print ()
    print ('Mean accuracy: \n', accuracy.mean())
    print ()
    
    print("Feature Importance: \n", )
    for index, i in enumerate(xgmodel.feature_importances_):
        print (f'Feature {index}: {i:.5f}')
    print()
    print("Model: \n",xgmodel)
    print()
    print("Accuracy score: \n", a_score)
    print()
    print("Probability: \n", probability)
    print()
    print("Confusion matrix: \n", c_matrix)
    print()
    print("Classification report: \n", c_report)
    

    return xgmodel, a_score, probability,c_matrix, c_report, x_train, y_train

In [11]:
result = do_xg(df, 'success',4)

[00:06:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:07:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\user\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[00:07:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:07:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:07:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[00:07:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [10]:
print(result[4])

              precision    recall  f1-score   support

         0.0       0.90      0.81      0.85     15103
         1.0       0.88      0.94      0.91     23407

    accuracy                           0.89     38510
   macro avg       0.89      0.88      0.88     38510
weighted avg       0.89      0.89      0.89     38510



In [14]:
print (result[3])

[[12182  2921]
 [ 1292 22115]]


# Grid search  for XG Boost

In [15]:
from sklearn.model_selection import GridSearchCV
import numpy as np

#find the parameter
parameters = {'max_depth': range(1, 5)}

#import grid search
grid_search= GridSearchCV(estimator= result[0], param_grid= parameters,scoring='accuracy',cv=10, n_jobs=-1)

#fit the train data in the grid search
Grid_search = grid_search.fit(x_train, y_train)

print(grid_search.best_score_)
print(grid_search.best_params_)

[00:12:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8846497124043381
{'max_depth': 4}


# The tree with max_depth 4 would give the best prediction 